# Мастерская Лента

## Описание задачи

Построить модель прогноза спроса на 14 дней для товаров собственного производства на основе мастер данных и данных продаж с учетом разных признаков. Гранулярность ТК-SKU-День.

Прогноз позволит повысить доступность и продажи в ТК, без повышения списаний и повышение маржинальности. При изготовлении товаров СП сотрудники будут ориентироваться не на экспертный подход, а на ML прогноз спроса, в соответствии с которым будут изготавливать продукцию и планировать заказы сырья.

## Знакомство с данными

Бизнес-заказчиком передано 3 датасета: `pr_df.csv`, `sales_df_train.csv`, `sales_submission.csv`:

1. Датасет `sales_df_train.csv` содержит данные по продажам за скользящий год для обучения.

    Столбцы:

    `st_id` – захэшированное id магазина;

    `pr_sku_id` – захэшированное id товара;

    `date` – дата;

    `pr_sales_type_id` – флаг наличия промо;

    `pr_sales_in_units` – число проданных товаров всего (промо и без промо);

    `pr_promo_sales_in_units` – число проданных товаров с признаком промо; 

    `pr_sales_in_rub` – продажи всего (промо и без промо) в РУБ; 

    `pr_promo_sales_in_rub` – продажи с признаком промо в РУБ;

2. Датасет `pr_df.csv` содержит данные по товарной иерархии. Иерархия представлена последовательностью от большего к меньшему `pr_group_id` - `pr_cat_id` - `pr_subcat_id` - `pr_sku_id`.

    Столбцы:

    `pr_group_id` – захэшированная группа товара;

    `pr_cat_id` – захэшированная категория товара;

    `pr_subcat_id` – захэшированная подкатегория товара;

    `pr_sku_id` – захэшированное id товара;

    `pr_uom_id` (маркер, обозначающий продаётся товар на вес или в ШТ).

3. Датасет `sf_df.csv` содержит данные по магазинам.

    Столбцы:

    `st_id` – захэшированное id магазина;

    `st_city_id` – захэшированное id города;

    `st_division_code id` – захэшированное id дивизиона; 
    
    `st_type_format_id` – id формата магазина;

    `st_type_loc_id` – id тип локации/окружения магазина; 
    
    `st_type_size_id` – id типа размера магазина;

    `st_is_active` – флаг активного магазина на данный момент.

Импортируем необходимые библиотеки:

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

Прочитаем представленные данные и посмотрим общую информацию о них:

In [83]:
pr_df = pd.read_csv('pr_df.csv')

In [84]:
sales_df_train = pd.read_csv('sales_df_train.csv')

In [85]:
sf_df = pd.read_csv('st_df.csv')

In [86]:
def read_data(data):
    print(data.info())
    print(data.head())
    print(data.describe())

In [87]:
read_data(pr_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pr_sku_id     2050 non-null   object
 1   pr_group_id   2050 non-null   object
 2   pr_cat_id     2050 non-null   object
 3   pr_subcat_id  2050 non-null   object
 4   pr_uom_id     2050 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 80.2+ KB
None
                          pr_sku_id                       pr_group_id  \
0  fd064933250b0bfe4f926b867b0a5ec8  c74d97b01eae257e44aa9d5bade97baf   
1  71c9661741caf40a92a32d1cc8206c04  c74d97b01eae257e44aa9d5bade97baf   
2  00b72c2f01a1512cbb1d3f33319bac93  c74d97b01eae257e44aa9d5bade97baf   
3  9bc40cd2fe4f188f402bb41548c5e15c  c74d97b01eae257e44aa9d5bade97baf   
4  3a74a370c8eb032acb11ad9119242b8f  c74d97b01eae257e44aa9d5bade97baf   

                          pr_cat_id                      pr_subcat_id  \
0  1bc0249a6412ef49b07fe6f62e6dc

Датасет указывает на иерархию товаров, пропусков в данных нет. Оставляем эти данные без изменений.

In [88]:
read_data(sales_df_train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883015 entries, 0 to 883014
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   date                     883015 non-null  object 
 3   pr_sales_type_id         883015 non-null  int64  
 4   pr_sales_in_units        883015 non-null  float64
 5   pr_promo_sales_in_units  883015 non-null  float64
 6   pr_sales_in_rub          883015 non-null  float64
 7   pr_promo_sales_in_rub    883015 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 53.9+ MB
None
                              st_id                         pr_sku_id  \
0  c81e728d9d4c2f636f067f89cc14862c  c7b711619071c92bef604c7ad68380dd   
1  42a0e188f5033bc65bf8d78622277c4e  68f441429e73dbd33b81f95da31576e9   
2  1ecfb463472ec9115b10c292ef8bc986  22988026fe85fde6

Сразу вызывают сомнения минусовые значения в столбцах количества и суммы продаж. Либо это возвраты, либо какие то ошибочные данные, в любом случае мы их убираем из наших данных. Аналогично поступим и с нулевыми значениями продаж, ведь нам нужно предугадать спрос, следовательно, нет смысла отталкиваться от нулевых значений.

In [89]:
read_data(sf_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   st_id              12 non-null     object
 1   st_city_id         12 non-null     object
 2   st_division_code   12 non-null     object
 3   st_type_format_id  12 non-null     int64 
 4   st_type_loc_id     12 non-null     int64 
 5   st_type_size_id    12 non-null     int64 
 6   st_is_active       12 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 800.0+ bytes
None
                              st_id                        st_city_id  \
0  1aa057313c28fa4a40c5bc084b11d276  1587965fb4d4b5afe8428a4a024feb0d   
1  bd470ca955d9497bbcb808e59952fffc  955d864a62659945cc9434898e275deb   
2  6364d3f0f495b6ab9dcf8d3b5c6e0b01  b8b4b727d6f5d1b61fff7be687f7970f   
3  1ecfb463472ec9115b10c292ef8bc986  908c9a564a86426585b29f5335b619bc   
4  16a5cdae362b8d27a1d8f8c7b78b4330  c1f75cc0f7fe2

В данном датасете у нас хранится информация о торговых комплексах (ТК). Интересный столбец со статусом магазина `st_is_active`. То есть наши данные хранят информацию о уже неактивных магазинах, их мы тоже смело можем исключить. Они в нашей модели будут лишними, ведь предсказание будет зависеть от ТК.

### Объединение данных

Для начала объединим наши данные в один датасет:

In [114]:
data = sales_df_train.merge(sf_df, on = 'st_id').merge(pr_df, on = 'pr_sku_id') 

In [103]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816401 entries, 0 to 816400
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    816401 non-null  object 
 1   pr_sku_id                816401 non-null  object 
 2   date                     816401 non-null  object 
 3   pr_sales_type_id         816401 non-null  int64  
 4   pr_sales_in_units        816401 non-null  float64
 5   pr_promo_sales_in_units  816401 non-null  float64
 6   pr_sales_in_rub          816401 non-null  float64
 7   pr_promo_sales_in_rub    816401 non-null  float64
 8   st_city_id               816401 non-null  object 
 9   st_division_code         816401 non-null  object 
 10  st_type_format_id        816401 non-null  int64  
 11  st_type_loc_id           816401 non-null  int64  
 12  st_type_size_id          816401 non-null  int64  
 13  st_is_active             816401 non-null  int64  
 14  pr_g

### Удаление продаж с минусовыми и нулевыми значениями 

In [106]:
data = data.drop(np.where(data['pr_sales_in_units'] <= 0)[0])

### Удаление неактивных ТК

В данных содержится информация о двух ТК, которые уже неактивны, информацию о данных ТК мы и удалим.

In [111]:
data = data.drop(np.where(data['st_is_active'] == 0)[0])

In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 815673 entries, 0 to 816400
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    815673 non-null  object 
 1   pr_sku_id                815673 non-null  object 
 2   date                     815673 non-null  object 
 3   pr_sales_type_id         815673 non-null  int64  
 4   pr_sales_in_units        815673 non-null  float64
 5   pr_promo_sales_in_units  815673 non-null  float64
 6   pr_sales_in_rub          815673 non-null  float64
 7   pr_promo_sales_in_rub    815673 non-null  float64
 8   st_city_id               815673 non-null  object 
 9   st_division_code         815673 non-null  object 
 10  st_type_format_id        815673 non-null  int64  
 11  st_type_loc_id           815673 non-null  int64  
 12  st_type_size_id          815673 non-null  int64  
 13  st_is_active             815673 non-null  int64  
 14  pr_group_

Изначальный набор данных содержал 883015 строк о продажах товаров собственного производства. После начальной обработки данных осталось 815673 строк, т.е. мы очистили 7,6% данных.

In [137]:
data['date'] = pd.to_datetime(data['date'])

In [141]:
data = data.sort_values('date')